In [1]:
#Import libraries for loading dataset from csv file
import pandas as pd
import sqlite3

#Import libraries for data analysis and visualisation.
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime

#Changes pyplot style to Seaborn for better visualisations
plt.style.use('seaborn-v0_8')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#Loads data from csv file to Pandas dataframe
ufc_events = pd.read_csv('/kaggle/input/mma-dataset-2023-ufc/ufc_event_data.csv', parse_dates = ['event_date'])
ufc_fights = pd.read_csv('/kaggle/input/mma-dataset-2023-ufc/ufc_fight_data.csv')
ufc_fighters = pd.read_csv('/kaggle/input/mma-dataset-2023-ufc/ufc_fighter_data.csv', parse_dates = ['fighter_dob'])
ufc_fight_stats = pd.read_csv('/kaggle/input/mma-dataset-2023-ufc/ufc_fight_stat_data.csv')
pd.options.display.max_columns = None

In [4]:
ufc_fighters.head()

,fighter_id,fighter_f_name,fighter_l_name,fighter_nickname,fighter_height_cm,fighter_weight_lbs,fighter_reach_cm,fighter_stance,fighter_dob,fighter_w,fighter_l,fighter_d,fighter_nc_dq,fighter_url
0,4107,Tom,Aaron,NaN,NaN,155.0,NaN,NaN,1978-07-13,5,3,0,NaN,http://ufcstats.com/fighter-details/93fe7332d1...
1,4106,Danny,Abbadi,The Assassin,180.34,155.0,NaN,Orthodox,1983-07-03,4,6,0,NaN,http://ufcstats.com/fighter-details/15df64c02b...
2,4105,Nariman,Abbasov,Bayraktar,172.72,155.0,167.64,Orthodox,1994-02-01,28,4,0,NaN,http://ufcstats.com/fighter-details/59a9d6dac6...
3,4104,David,Abbott,Tank,182.88,265.0,NaN,Switch,NaT,10,15,0,NaN,http://ufcstats.com/fighter-details/b361180739...
4,4103,Hamdy,Abdelwahab,The Hammer,187.96,264.0,182.88,Southpaw,1993-01-22,5,0,0,1.0,http://ufcstats.com/fighter-details/3329d692ae...


In [7]:
events_date = ufc_events[['event_id','event_date']]
fights_data = ufc_fights.copy()
fights_data.head()

,fight_id,event_id,referee,f_1,f_2,winner,num_rounds,title_fight,weight_class,gender,result,result_details,finish_round,finish_time,fight_url
0,7218,664,Herb Dean,2976.0,2884.0,2884.0,5,F,Lightweight,M,KO/TKO,to \n Leg Injury,2,2:03,http://ufcstats.com/fight-details/23a604f46028...
1,7217,664,Mark Smith,1662.0,2464.0,1662.0,3,F,Featherweight,M,Decision,Unanimous,3,5:00,http://ufcstats.com/fight-details/da1b37edb8cc...
2,7216,664,Kerry Hatley,981.0,179.0,981.0,3,F,Women's Strawweight,F,KO/TKO,Punches to Head From Mount,2,2:42,http://ufcstats.com/fight-details/d8335b728604...
3,7215,664,Dan Miragliotta,3831.0,2974.0,3831.0,3,F,Welterweight,M,Submission,Rear Naked Choke,2,4:32,http://ufcstats.com/fight-details/bf647be41de3...
4,7214,664,Herb Dean,1108.0,2320.0,2320.0,3,F,Featherweight,M,Submission,Guillotine Choke From Bottom Guard,1,3:12,http://ufcstats.com/fight-details/6e1bf1b163b3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7213,5,1,John McCarthy,141.0,1962.0,141.0,N,F,Open Weight,M,KO/TKO,to \n Corner Stoppage,1,2:50,http://ufcstats.com/fight-details/3b020d4914b4...
7214,4,1,John McCarthy,2663.0,1952.0,2663.0,N,F,Open Weight,M,Submission,Keylock From Half Guard,1,4:52,http://ufcstats.com/fight-details/d917c8c7461b...
7215,3,1,John McCarthy,1060.0,2038.0,1060.0,N,F,Open Weight,M,KO/TKO,Punches to Head From Guard\n Submission t...,1,2:13,http://ufcstats.com/fight-details/ccee020be2e8...
7216,2,1,John McCarthy,640.0,108.0,640.0,N,F,Open Weight,M,Submission,Guillotine Choke Standing,1,0:58,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [ ]:
events_date.head()

In [8]:
fights_data = pd.merge(fights_data, events_date, how='left', on='event_id')

In [9]:
fights_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7218 entries, 0 to 7217
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   fight_id        7218 non-null   int64         
 1   event_id        7218 non-null   int64         
 2   referee         7186 non-null   object        
 3   f_1             7199 non-null   float64       
 4   f_2             7205 non-null   float64       
 5   winner          7203 non-null   float64       
 6   num_rounds      7218 non-null   object        
 7   title_fight     7218 non-null   object        
 8   weight_class    7205 non-null   object        
 9   gender          7218 non-null   object        
 10  result          7218 non-null   object        
 11  result_details  7201 non-null   object        
 12  finish_round    7218 non-null   int64         
 13  finish_time     7218 non-null   object        
 14  fight_url       7218 non-null   object        
 15  even

In [10]:
#add date from ufc_events to ufc_fights
fights_data = fights_data.drop(['referee','fight_url','result_details','event_id'], axis=1)
fights_data.head()

,fight_id,f_1,f_2,winner,num_rounds,title_fight,weight_class,gender,result,finish_round,finish_time,event_date
0,7218,2976.0,2884.0,2884.0,5,F,Lightweight,M,KO/TKO,2,2:03,2023-09-23
1,7217,1662.0,2464.0,1662.0,3,F,Featherweight,M,Decision,3,5:00,2023-09-23
2,7216,981.0,179.0,981.0,3,F,Women's Strawweight,F,KO/TKO,2,2:42,2023-09-23
3,7215,3831.0,2974.0,3831.0,3,F,Welterweight,M,Submission,2,4:32,2023-09-23
4,7214,1108.0,2320.0,2320.0,3,F,Featherweight,M,Submission,1,3:12,2023-09-23


In [13]:
fights_data.result.unique()

array(['KO/TKO', 'Decision', 'Submission', 'DQ',
       "TKO - Doctor's Stoppage"], dtype=object)

In [ ]:
d1 = datetime.datetime(2018, 5, 3)
cond = fights_data['event_date'][1000] < fights_data['event_date'][0]
print(d1)
print(cond)
print(cond)

In [ ]:
#Create function who return fighters past experience
def fighter_xp(first_name, last_name, date):
    f_infos = ufc_fighters.loc[(ufc_fighters.fighter_f_name == first_name) & (ufc_fighters.fighter_l_name == last_name)]
    f_fights = ufc_fights.loc[(fights_data.f_1 == fighter.fighter_id) | (fights_data.f_2 == fighter.fighter_id)]
    tot_wins = 0
    tot_loses = 0
    tot_draws = 0
    for i in fights_data:
        if fights_data['event_date'][i] < date :
            if fights_data['f_1'][i] == f_infos.fighter_id | fights_data['f_2'][i] == f_infos.fighter_id :
                if fights_data['winner'][i] == f_infos.fighter_id :
                    tot_wins += 1
                elif fights_data['winner'][i] == fights_data['f_1'][i] | fights_data['winner'][i] == fights_data['f_2'][i]:
                    tot_loses += 1
                else :
                    tot_draws += 1
    return fighter_xp

In [ ]:
fights_data.loc[fights_data.f_1 == ]

In [12]:
def fighter_name(id):
    return ufc_fighters.loc[ufc_fighters.fighter_id == id].fighter_f_name + ' ' + ufc_fighters.loc[ufc_fighters.fighter_id == id].fighter_l_name